Написать функцию, которая будет добавлять в вашу базу данных только новые вакансии с сайта.

In [3]:
from bs4 import BeautifulSoup as bs  
from bs4 import BeautifulStoneSoup    
import bs4     
import requests
import json
import pandas as pd
import numpy as np
import re
from pymongo import MongoClient
import ast


In [4]:
client = MongoClient('localhost', 27017)
db = client['HH_database']
collection = db.HH_API_collection

In [5]:
def getPage(page = 0):
    """
    Создаем метод для получения страницы со списком вакансий.
    Аргументы:
        page - Индекс страницы, начинается с 0. Значение по умолчанию 0, т.е. первая страница
    """
     
    # Справочник для параметров GET-запроса
    params = {
        'text': 'NAME:Data Scientist', # Текст фильтра. В имени должно быть слово "Аналитик"
        'area': 1, # Поиск ощуществляется по вакансиям города Москва
        'page': page, # Индекс страницы поиска на HH
        'per_page': 100 # Кол-во вакансий на 1 странице
    }
     
     
    req = requests.get('https://api.hh.ru/vacancies', params) # Посылаем запрос к API
    data = req.content.decode() # Декодируем его ответ, чтобы Кириллица отображалась корректно
    req.close()
    return data

In [6]:
def load_data(p):
    jsObj = json.loads(getPage(0))
    my_df = pd.DataFrame.from_dict(jsObj['items'], orient='columns')
    # Считываем первые 2000 вакансий
    for page in range(1, 20):

        # Преобразуем текст ответа запроса в справочник Python
        jsObj = json.loads(getPage(page))
        my_df_ = pd.DataFrame.from_dict(jsObj['items'], orient='columns')
        my_df = pd.concat([my_df, my_df_], axis=0)
        # Проверка на последнюю страницу, если вакансий меньше 2000
        if (jsObj['pages'] - page) <= 1:
            break

    print('Старницы поиска собраны')
    return my_df

In [7]:
my_df = load_data(1)

Старницы поиска собраны


In [8]:
for i, row in my_df.iterrows():
    if row['salary'] != None:
        for e in row['salary']:
            my_df.loc[i, e] = row['salary'][e]
my_df.rename(columns={'from': 'salary_from', 'to':'salary_to', 'currency':'salary_currency', 'gross':'salary_gross'}, inplace=True)

In [ ]:
Если вакансия есть, обновляем зарплату(вдруг подросла), если нет, создаем новую запись

In [11]:
for i, row in my_df.iterrows():
    #print(i,' **** ', row)
    comparison_target = row['employer']
    db.collection.update_one(
        {'employer': comparison_target, 'name':row['name']},
        {'$set': {'salary': row['salary'], 'salary_from': row['salary_from'], 'salary_to': row['salary_to']}}, 
        upsert=True
    )